In [4]:
import numpy as np
from scipy.stats import levy

In [22]:
class FPA():
    def __init__(self, switch_probability=0.8, n_flowers=50, bounds=None):
        # Initialization of variables
        self.sp = switch_probability
        self.n_flowers = n_flowers
        self.flowers = [None] * n_flowers
        self.cost = np.zeros(n_flowers)
        self.random = np.random
        self.n_parameters = len(bounds)
        self.bounds = bounds

        # Random Initial Flowers
        self.init_flowers()
        # Get the best flower from initial population
        self.best = self.flowers[self.cost.argmin()]

    def Six_Hump_Camel(self, x):
        # This is the objective function, modify this according to your needs.
        output = (1 + 2 * x[0] + (3 * x[1] - 1) + 3 * x[2] + 2 * x[3] ** 2 + (5 * x[4] + 2))
        return output

    def global_pollination(self, x):
        # Global pollination
        x_new = x + self.random.normal(size=x.shape) * (self.best - x)
        return x_new

    def local_pollination(self, x, x1, x2):
        # Local pollination
        x_new = x + self.random.normal(size=x.shape) * (x1 - x2)
        return x_new

    def init_flowers(self):
        # Initialization of flowers within bounds
        for i in range(self.n_flowers):
            self.flowers[i] = np.array([self.random.uniform(low=bound[0], high=bound[1]) for bound in self.bounds])
            self.cost[i] = self.Six_Hump_Camel(self.flowers[i])

    def optimize(self, max_gen=100, convergence_threshold=5):

      # Initialize convergence counter
      convergence_counter = 0

      # Previous best cost
      prev_best_cost = np.inf

      # Generation loop
      for i in range(max_gen):
          # Flower loop
          for j in range(self.n_flowers):
              p = self.random.rand()

              # Global Pollination if p <= switch probability
              if p <= self.sp:
                  x_temp = self.global_pollination(self.flowers[j])

              # Local Pollination if p > switch probability
              else:
                  r1 = self.random.randint(0, high=self.n_flowers)
                  r2 = self.random.randint(0, high=self.n_flowers)
                  while r2 == r1:
                      r2 = self.random.randint(0, high=self.n_flowers)
                  x_temp = self.local_pollination(self.flowers[j], self.flowers[r1], self.flowers[r2])

              # Apply bounds
              x_temp = np.clip(x_temp, [bound[0] for bound in self.bounds], [bound[1] for bound in self.bounds])

              # Calculate cost
              cost_temp = self.Six_Hump_Camel(x_temp)

              # Compare the newly generated flower with the previous flower
              if cost_temp < self.cost[j]:
                  self.flowers[j] = x_temp
                  self.cost[j] = cost_temp
              else:
                  continue

          # Update best
          self.best = self.flowers[self.cost.argmin()]

          # Print generation number and minima points
          print(f"Generation {i + 1}: Minimum point: {self.best}, Cost: {self.cost.min()}")

          # Check for convergence
          if np.abs(prev_best_cost - self.cost.min()) < 1e-6:
              convergence_counter += 1
              if convergence_counter >= convergence_threshold:
                  print(f"Convergence reached with minimum cost: {self.cost.min()}")
                  break
          else:
              convergence_counter = 0  # Reset convergence counter
              prev_best_cost = self.cost.min()

      return self.flowers, self.cost


In [23]:
bounds = [(10, 60), (15, 30), (25, 75), (10, 30), (10, 50)]
FPA_instance = FPA(switch_probability=0.6, n_flowers=5, bounds=bounds)
result, cost = FPA_instance.optimize(25)

Generation 1: Minimum point: [24.03046635 23.43455406 50.29760217 12.22592039 12.80355848], Cost: 634.2214524659925
Generation 2: Minimum point: [60.         17.7750716  66.34809139 10.         10.        ], Cost: 624.3694889788017
Generation 3: Minimum point: [60.         17.7750716  66.34809139 10.         10.        ], Cost: 624.3694889788017
Generation 4: Minimum point: [50.80542206 15.         46.73140746 10.         10.        ], Cost: 538.8050664782927
Generation 5: Minimum point: [50.80542206 15.         46.73140746 10.         10.        ], Cost: 538.8050664782927
Generation 6: Minimum point: [10. 30. 25. 10. 10.], Cost: 437.0
Generation 7: Minimum point: [10. 30. 25. 10. 10.], Cost: 437.0
Generation 8: Minimum point: [10.         22.62373227 25.         10.         10.        ], Cost: 414.87119680059834
Generation 9: Minimum point: [10.         22.62373227 25.         10.         10.        ], Cost: 414.87119680059834
Generation 10: Minimum point: [10.         22.62373227 25.